# Materialized Views

This notebook demonstrates how to effectively use materialized views in PostgreSQL:
* Creating materialized views
* Refresh strategies
* Performance comparisons
* Best practices

## 1. Creating Basic Materialized Views

In [ ]:
-- Create materialized view for daily sales metrics
CREATE MATERIALIZED VIEW daily_sales_metrics AS
SELECT 
    DATE_TRUNC('day', o.order_date) as sale_date,
    p.category,
    COUNT(DISTINCT o.order_id) as num_orders,
    COUNT(DISTINCT o.customer_id) as num_customers,
    SUM(oi.quantity) as total_items,
    SUM(oi.quantity * oi.unit_price) as total_revenue
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE o.status = 'Completed'
GROUP BY DATE_TRUNC('day', o.order_date), p.category
WITH DATA;

## 2. Comparing Performance

In [ ]:
-- Original query (without materialized view)
EXPLAIN ANALYZE
SELECT 
    DATE_TRUNC('day', o.order_date) as sale_date,
    p.category,
    COUNT(DISTINCT o.order_id) as num_orders,
    COUNT(DISTINCT o.customer_id) as num_customers,
    SUM(oi.quantity) as total_items,
    SUM(oi.quantity * oi.unit_price) as total_revenue
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE o.status = 'Completed'
AND o.order_date >= '2022-01-01'
GROUP BY DATE_TRUNC('day', o.order_date), p.category;

-- Query using materialized view
EXPLAIN ANALYZE
SELECT *
FROM daily_sales_metrics
WHERE sale_date >= '2022-01-01';

## 3. Refresh Strategies

In [ ]:
-- Complete refresh
REFRESH MATERIALIZED VIEW daily_sales_metrics;

-- Create materialized view that supports concurrent refresh
CREATE MATERIALIZED VIEW customer_metrics
WITH (fillfactor=70) AS
SELECT 
    c.customer_id,
    c.country,
    c.segment,
    COUNT(o.order_id) as total_orders,
    SUM(o.total_amount) as total_spent,
    MAX(o.order_date) as last_order_date,
    AVG(o.total_amount) as avg_order_value
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.country, c.segment
WITH DATA;

-- Create unique index to support concurrent refresh
CREATE UNIQUE INDEX idx_customer_metrics_id ON customer_metrics(customer_id);

-- Concurrent refresh
REFRESH MATERIALIZED VIEW CONCURRENTLY customer_metrics;

## 4. Complex Materialized Views

In [ ]:
-- Create materialized view for product performance analysis
CREATE MATERIALIZED VIEW product_performance AS
WITH monthly_sales AS (
    SELECT 
        p.product_id,
        p.product_name,
        p.category,
        DATE_TRUNC('month', o.order_date) as sale_month,
        SUM(oi.quantity) as units_sold,
        SUM(oi.quantity * oi.unit_price) as revenue,
        COUNT(DISTINCT o.order_id) as num_orders
    FROM products p
    JOIN order_items oi ON p.product_id = oi.product_id
    JOIN orders o ON oi.order_id = o.order_id
    WHERE o.status = 'Completed'
    GROUP BY p.product_id, p.product_name, p.category, DATE_TRUNC('month', o.order_date)
)
SELECT 
    product_id,
    product_name,
    category,
    sale_month,
    units_sold,
    revenue,
    num_orders,
    LAG(units_sold) OVER (PARTITION BY product_id ORDER BY sale_month) as prev_month_units,
    LAG(revenue) OVER (PARTITION BY product_id ORDER BY sale_month) as prev_month_revenue
FROM monthly_sales
WITH DATA;

-- Create indexes for common query patterns
CREATE INDEX idx_product_perf_date ON product_performance(sale_month);
CREATE INDEX idx_product_perf_category ON product_performance(category);

## 5. Maintenance and Monitoring

In [ ]:
-- Check materialized view status
SELECT 
    schemaname,
    matviewname,
    matviewowner,
    ispopulated,
    pg_size_pretty(pg_relation_size(schemaname || '.' || matviewname::regclass)) as size
FROM pg_matviews;

-- Check when materialized views were last refreshed
SELECT 
    schemaname,
    matviewname,
    pg_size_pretty(pg_relation_size(schemaname || '.' || matviewname::regclass)) as size,
    pg_stat_get_last_analyze_time(schemaname || '.' || matviewname::regclass) as last_analyzed,
    pg_stat_get_last_autoanalyze_time(schemaname || '.' || matviewname::regclass) as last_autoanalyzed
FROM pg_matviews;

## Best Practices for Materialized Views

1. **When to Use Materialized Views**
   - Complex queries that are frequently executed
   - Aggregations over large datasets
   - Reports that don't need real-time data
   - Data that changes infrequently

2. **Refresh Strategy**
   - Schedule refreshes during off-peak hours
   - Use concurrent refresh when possible
   - Consider refresh frequency vs. data freshness
   - Monitor refresh performance

3. **Design Considerations**
   - Create appropriate indexes
   - Consider storage requirements
   - Plan for maintenance windows
   - Use unique indexes for concurrent refresh

4. **Common Pitfalls**
   - Refreshing too frequently
   - Not maintaining statistics
   - Missing indexes on materialized views
   - Ignoring storage costs